<a href="https://colab.research.google.com/github/bikramgit/notebooks/blob/main/rag_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


In [ ]:
%pip install -qU "langchain[google-genai]"

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ··········


In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
%pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
%pip install -qU langchain-community

In [ ]:
%pip install -qU langchain-hub
%pip install -U langchain-core langchain-prompts


ERROR: Could not find a version that satisfies the requirement langchain-hub (from versions: none)
ERROR: No matching distribution found for langchain-hub
ERROR: Could not find a version that satisfies the requirement langchain-prompts (from versions: none)
ERROR: No matching distribution found for langchain-prompts


In [ ]:
import bs4
from langchainhub import Client
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.prompts import ChatPromptTemplate

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# 🧠 Assume you have an initialized vector store and LLM here
# For example:
# from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# vector_store = FAISS.from_documents(all_splits, OpenAIEmbeddings())
# llm = ChatOpenAI(model="gpt-4o-mini")

# ✅ Use the new LangChainHub client
hub = Client()
prompt_text = hub.pull("rlm/rag-prompt")
prompt = ChatPromptTemplate.from_template(prompt_text)

prompt = ChatPromptTemplate.from_template("""
Answer the question below:
ID: {id}
Question: {question}
Context: {context}
""")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content, "id": "001"})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()


/tmp/ipython-input-1310301242.py:33: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  prompt_text = hub.pull("rlm/rag-prompt")


In [ ]:
print(prompt.input_variables)
# #response = graph.invoke({"question": "What is Task Decomposition?"})
# messages = prompt.invoke({
#     '"id"': "001",  # notice quotes inside key
#     "question": state["question": "What is Task Decomposition?"],
#     "context": docs_content
# })
# print(messages["answer"])

# # Prepare your question and context manually
# question = "What is Task Decomposition?"
# docs_content = "\n\n".join(doc.page_content for doc in all_splits[:3])  # sample chunks

# # Invoke the prompt correctly
# messages = prompt.invoke({
#     "id": "001",          # satisfies weird template variable
#     "question": question,
#     "context": docs_content
# })

# # Show the generated prompt text
# print(messages)
# print(prompt.input_variables)
# response = graph.invoke({"id": "001", "question": "What is Task Decomposition?"})
# print(response["answer"])

docs_content = "\n\n".join(doc.page_content for doc in all_splits[:3])  # sample chunks

# Call graph.invoke with all required variables
response = graph.invoke({
    "id": "001",                    # required by prompt
    "question": "What is Task Decomposition?",
    "context": docs_content         # the retrieved context for RAG
})

# Print the answer
print(response["answer"])

['context', 'id', 'question']
Task Decomposition is the process of breaking down a large, complex task into smaller, more manageable, and often interdependent sub-tasks or components.

**Key aspects:**
*   **Hierarchical Structure:** It typically involves creating a hierarchical structure, where a main task is broken into several sub-tasks, and those sub-tasks might be further broken down into even smaller units, until each unit is specific, actionable, and easy to understand.
*   **Reduced Complexity:** The primary goal is to simplify a daunting or ambiguous task into distinct, well-defined pieces, making it easier to comprehend, plan, and execute.

**Benefits of Task Decomposition:**
*   **Improved Understanding:** Makes complex problems easier to grasp.
*   **Better Planning & Estimation:** Allows for more accurate time, resource, and budget estimations for each sub-task.
*   **Easier Management:** Facilitates the assignment of specific sub-tasks to individuals or teams.
*   **Enhan